<a href="https://colab.research.google.com/github/andgreenman/OpenAlex-to-Scopus-crosswalk/blob/main/Crosswalk_OpenAlex_to_Scopus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script takes an exported search results list from the OpenAlex website, which cannot normally be loaded into VOSViewer directly, and coverts it to the format of a Scopus export. Custom filtering of the data exported from OpenAlex should be performed before this step.

In [ ]:
pip install requests-cache

In [ ]:
import datetime
import pandas as pd
import json
import requests
import requests_cache
from requests.exceptions import HTTPError

In [ ]:
file = r'' #put csv file path string here

In [ ]:
works = pd.read_csv(file, header=0)
df = pd.DataFrame(works)
df.dropna(subset='publication_year') # OpenAlex exports sometimes have random junk rows, usually this catches them

In [ ]:
take_columns = ['authorships.author.display_name',
                'authorships.author.id',
                'title',
                'publication_year',
                'primary_location.source.display_name',
                'biblio.volume',
                'biblio.issue',
                'biblio.first_page',
                'biblio.last_page',
                'cited_by_count',
                'doi',
                'ids.openalex',
                'authorships.raw_affiliation_strings',
                'abstract',
                'topics.display_name',
                'keywords.display_name',
                'grants.funder_display_name',
                'referenced_works',
                'primary_location.source.host_organization_name',
                'primary_location.source.issn_l',
                'ids.pmcid',
                'language',
                'type',
                'primary_location.version',
                'open_access.oa_status',
                'id']

scopus_columns = ['Authors',
                'Author(s) ID',
                'Title',
                'Year',
                'Source title',
                'Volume',
                'Issue',
                'Page start',
                'Page end',
                'Cited by',
                'DOI',
                'Link',
                'Affiliations',
                'Abstract',
                'Author Keywords',
                'Index Keywords',
                'Funding Details',
                'References',
                'Publisher',
                'ISSN',
                'PubMed ID',
                'Language of Original Document',
                'Document Type',
                'Publication Stage',
                'Open Access',
                'EID']

rename_dict = dict(zip(take_columns, scopus_columns))

In [ ]:
dfscopus = df.loc[:, take_columns]

In [ ]:
dfscopus.rename(columns=rename_dict, inplace=True)
df.columns = df.columns.str.strip()

In [ ]:
columns_to_modify = ['Authors', 'Author(s) ID', 'Affiliations', 'Author Keywords', 'Index Keywords', 'Funding Details']

In [ ]:
dfscopus[columns_to_modify] = dfscopus[columns_to_modify].apply(lambda x: x.str.replace(r'\|', '; ', regex=True))

dfscopus['DOI'] = dfscopus['DOI'].str.removeprefix('https://doi.org/')

dfscopus['EID'] = dfscopus['EID'].str.removeprefix('https://openalex.org/')

dfscopus['Author(s) ID'] = dfscopus['Author(s) ID'].apply(lambda ids: ';'.join([id.replace('https://openalex.org/', '') for id in ids.split(';')]))

dfscopus['References'] = dfscopus['References'].apply(lambda references: '|'.join([reference.replace('https://openalex.org/', '') for reference in str(references).split('|')]) if not isinstance(references, float) else references)

In [ ]:
dfscopus.insert(1, 'Author full names', '')
dfscopus.insert(8, 'Art. No.', '')
dfscopus.insert(11, 'Page count', '')
dfscopus.insert(16, 'Authors with affiliations', '')
dfscopus.insert(21, 'Funding Texts', '')
dfscopus.insert(23, 'Correspondence Address', '')
dfscopus.insert(24, 'Editors', '')
dfscopus.insert(27, 'ISBN', '')
dfscopus.insert(28, 'CODEN', '')
dfscopus.insert(31, 'Abbreviated Source Title', '')
dfscopus.insert(35, 'Source', 'OpenAlex')

In [ ]:
def name_fl_to_li(name):
  name_parts = name.split(' ')
  temp = '.'.join([name_parts[i][0] for i in range (0, len(name_parts) - 1)])
  if temp == '':
      return (name_parts[-1])
  else:
      return (name_parts[-1] + ', ' + temp +'.')

In [ ]:
def process_authors(authors):
    author_list = authors.split(';')
    formatted_authors = []
    for author in author_list:
        author = author.strip()
        formatted_authors.append(name_fl_to_li(author))

    return '; '.join(formatted_authors)

In [ ]:
dfscopus['Authors'] = dfscopus['Authors'].apply(lambda x: process_authors(x) if pd.notnull(x) else '')

In [ ]:
# coding: utf8

languages = [
    ('aa', 'Afar'),
    ('ab', 'Abkhazian'),
    ('af', 'Afrikaans'),
    ('ak', 'Akan'),
    ('sq', 'Albanian'),
    ('am', 'Amharic'),
    ('ar', 'Arabic'),
    ('an', 'Aragonese'),
    ('hy', 'Armenian'),
    ('as', 'Assamese'),
    ('av', 'Avaric'),
    ('ae', 'Avestan'),
    ('ay', 'Aymara'),
    ('az', 'Azerbaijani'),
    ('ba', 'Bashkir'),
    ('bm', 'Bambara'),
    ('eu', 'Basque'),
    ('be', 'Belarusian'),
    ('bn', 'Bengali'),
    ('bh', 'Bihari languages'),
    ('bi', 'Bislama'),
    ('bo', 'Tibetan'),
    ('bs', 'Bosnian'),
    ('br', 'Breton'),
    ('bg', 'Bulgarian'),
    ('my', 'Burmese'),
    ('ca', 'Catalan; Valencian'),
    ('cs', 'Czech'),
    ('ch', 'Chamorro'),
    ('ce', 'Chechen'),
    ('zh', 'Chinese'),
    ('cu', 'Church Slavic; Old Slavonic; Church Slavonic; Old Bulgarian; Old Church Slavonic'),
    ('cv', 'Chuvash'),
    ('kw', 'Cornish'),
    ('co', 'Corsican'),
    ('cr', 'Cree'),
    ('cy', 'Welsh'),
    ('cs', 'Czech'),
    ('da', 'Danish'),
    ('de', 'German'),
    ('dv', 'Divehi; Dhivehi; Maldivian'),
    ('nl', 'Dutch; Flemish'),
    ('dz', 'Dzongkha'),
    ('el', 'Greek, Modern (1453-)'),
    ('en', 'English'),
    ('eo', 'Esperanto'),
    ('et', 'Estonian'),
    ('eu', 'Basque'),
    ('ee', 'Ewe'),
    ('fo', 'Faroese'),
    ('fa', 'Persian'),
    ('fj', 'Fijian'),
    ('fi', 'Finnish'),
    ('fr', 'French'),
    ('fy', 'Western Frisian'),
    ('ff', 'Fulah'),
    ('Ga', 'Georgian'),
    ('de', 'German'),
    ('gd', 'Gaelic; Scottish Gaelic'),
    ('ga', 'Irish'),
    ('gl', 'Galician'),
    ('gv', 'Manx'),
    ('el', 'Greek, Modern (1453-)'),
    ('gn', 'Guarani'),
    ('gu', 'Gujarati'),
    ('ht', 'Haitian; Haitian Creole'),
    ('ha', 'Hausa'),
    ('he', 'Hebrew'),
    ('hz', 'Herero'),
    ('hi', 'Hindi'),
    ('ho', 'Hiri Motu'),
    ('hr', 'Croatian'),
    ('hu', 'Hungarian'),
    ('hy', 'Armenian'),
    ('ig', 'Igbo'),
    ('is', 'Icelandic'),
    ('io', 'Ido'),
    ('ii', 'Sichuan Yi; Nuosu'),
    ('iu', 'Inuktitut'),
    ('ie', 'Interlingue; Occidental'),
    ('ia', 'Interlingua (International Auxiliary Language Association)'),
    ('id', 'Indonesian'),
    ('ik', 'Inupiaq'),
    ('is', 'Icelandic'),
    ('it', 'Italian'),
    ('jv', 'Javanese'),
    ('ja', 'Japanese'),
    ('kl', 'Kalaallisut; Greenlandic'),
    ('kn', 'Kannada'),
    ('ks', 'Kashmiri'),
    ('ka', 'Georgian'),
    ('kr', 'Kanuri'),
    ('kk', 'Kazakh'),
    ('km', 'Central Khmer'),
    ('ki', 'Kikuyu; Gikuyu'),
    ('rw', 'Kinyarwanda'),
    ('ky', 'Kirghiz; Kyrgyz'),
    ('kv', 'Komi'),
    ('kg', 'Kongo'),
    ('ko', 'Korean'),
    ('kj', 'Kuanyama; Kwanyama'),
    ('ku', 'Kurdish'),
    ('lo', 'Lao'),
    ('la', 'Latin'),
    ('lv', 'Latvian'),
    ('li', 'Limburgan; Limburger; Limburgish'),
    ('ln', 'Lingala'),
    ('lt', 'Lithuanian'),
    ('lb', 'Luxembourgish; Letzeburgesch'),
    ('lu', 'Luba-Katanga'),
    ('lg', 'Ganda'),
    ('mk', 'Macedonian'),
    ('mh', 'Marshallese'),
    ('ml', 'Malayalam'),
    ('mi', 'Maori'),
    ('mr', 'Marathi'),
    ('ms', 'Malay'),
    ('Mi', 'Micmac'),
    ('mk', 'Macedonian'),
    ('mg', 'Malagasy'),
    ('mt', 'Maltese'),
    ('mn', 'Mongolian'),
    ('mi', 'Maori'),
    ('ms', 'Malay'),
    ('my', 'Burmese'),
    ('na', 'Nauru'),
    ('nv', 'Navajo; Navaho'),
    ('nr', 'Ndebele, South; South Ndebele'),
    ('nd', 'Ndebele, North; North Ndebele'),
    ('ng', 'Ndonga'),
    ('ne', 'Nepali'),
    ('nl', 'Dutch; Flemish'),
    ('nn', 'Norwegian Nynorsk; Nynorsk, Norwegian'),
    ('nb', 'Bokmål, Norwegian; Norwegian Bokmål'),
    ('no', 'Norwegian'),
    ('oc', 'Occitan (post 1500)'),
    ('oj', 'Ojibwa'),
    ('or', 'Oriya'),
    ('om', 'Oromo'),
    ('os', 'Ossetian; Ossetic'),
    ('pa', 'Panjabi; Punjabi'),
    ('fa', 'Persian'),
    ('pi', 'Pali'),
    ('pl', 'Polish'),
    ('pt', 'Portuguese'),
    ('ps', 'Pushto; Pashto'),
    ('qu', 'Quechua'),
    ('rm', 'Romansh'),
    ('ro', 'Romanian; Moldavian; Moldovan'),
    ('ro', 'Romanian; Moldavian; Moldovan'),
    ('rn', 'Rundi'),
    ('ru', 'Russian'),
    ('sg', 'Sango'),
    ('sa', 'Sanskrit'),
    ('si', 'Sinhala; Sinhalese'),
    ('sk', 'Slovak'),
    ('sk', 'Slovak'),
    ('sl', 'Slovenian'),
    ('se', 'Northern Sami'),
    ('sm', 'Samoan'),
    ('sn', 'Shona'),
    ('sd', 'Sindhi'),
    ('so', 'Somali'),
    ('st', 'Sotho, Southern'),
    ('es', 'Spanish; Castilian'),
    ('sq', 'Albanian'),
    ('sc', 'Sardinian'),
    ('sr', 'Serbian'),
    ('ss', 'Swati'),
    ('su', 'Sundanese'),
    ('sw', 'Swahili'),
    ('sv', 'Swedish'),
    ('ty', 'Tahitian'),
    ('ta', 'Tamil'),
    ('tt', 'Tatar'),
    ('te', 'Telugu'),
    ('tg', 'Tajik'),
    ('tl', 'Tagalog'),
    ('th', 'Thai'),
    ('bo', 'Tibetan'),
    ('ti', 'Tigrinya'),
    ('to', 'Tonga (Tonga Islands)'),
    ('tn', 'Tswana'),
    ('ts', 'Tsonga'),
    ('tk', 'Turkmen'),
    ('tr', 'Turkish'),
    ('tw', 'Twi'),
    ('ug', 'Uighur; Uyghur'),
    ('uk', 'Ukrainian'),
    ('ur', 'Urdu'),
    ('uz', 'Uzbek'),
    ('ve', 'Venda'),
    ('vi', 'Vietnamese'),
    ('vo', 'Volapük'),
    ('cy', 'Welsh'),
    ('wa', 'Walloon'),
    ('wo', 'Wolof'),
    ('xh', 'Xhosa'),
    ('yi', 'Yiddish'),
    ('yo', 'Yoruba'),
    ('za', 'Zhuang; Chuang'),
    ('zh', 'Chinese'),
    ('zu', 'Zulu')
]

language_dict = dict(languages)

In [ ]:
dfscopus.loc[:, 'Language of Original Document'] = dfscopus['Language of Original Document'].map(language_dict)

In [ ]:
s = requests_cache.CachedSession()

In [ ]:
def fetch_batched_references(referenced_works, batch_size=50):
    if pd.isna(referenced_works):
        return 'No references found'

    work_ids = referenced_works.split('|')
    batches = [work_ids[i:i + batch_size] for i in range(0, len(work_ids), batch_size)]

    details_list = []

    for batch in batches:
        batch_query = '|'.join(batch)
        url = f"https://api.openalex.org/works?filter=openalex:{batch_query}&per-page=50"

        try:
            response = s.get(url)
            response.raise_for_status()
            works_data = response.json().get('results', [])

            for work in works_data:
                authors = ', '.join([process_authors(auth['author']['display_name']) for auth in work.get('authorships', [])])
                title = work.get('title', 'No title found')
                publication_title = work.get('primary_location', {}).get('source', {}).get('display_name', 'No publication title found')
                publication_date = work.get('publication_year', 'No publication date found')
                details_list.append(f"{authors}, {title}, {publication_title}, {publication_date}")

        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred: {http_err} for batch: {batch_query}")
        except Exception as err:
            print(f"Other error occurred: {err}")

    return '; '.join(details_list)

In [ ]:
dfscopus['References'] = dfscopus['References'].apply(fetch_batched_references)

In [ ]:
filename = '' #put filename here

date = datetime.datetime.today().strftime('%Y-%m-%d')

In [ ]:
dfscopus.to_csv(date + '_' + filename, index=False)